In [ ]:
#coregistrations
#%pip install mne
#%pip install darkdetect
#see https://mne.tools/stable/auto_tutorials/forward/25_automated_coreg.html

#This code automates the creation of the head<->MRI transformation matrix based on fiducial alignment and ICP fitting
import mne
import os
import shutil
from mne.io import read_info
from mne.coreg import Coregistration
import mne.viz
mne.viz.set_3d_backend("pyvistaqt")
mne.viz.set_3d_options(antialias=False)
import numpy as np
mri_dir_general = r"D:\REFTEP_ALL\REFTEP_reco"
mri_dirs_general = os.listdir(mri_dir_general)
fiducials = 'estimated'
view_kwargs1 = dict(azimuth=45, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs2 = dict(azimuth=-45, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs3 = dict(azimuth=90, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs4 = dict(azimuth=-90, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs5 = dict(azimuth=135, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs6 = dict(azimuth=-135, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs7 = dict(azimuth=180, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))
view_kwargs_all = [view_kwargs1, view_kwargs2, view_kwargs3, view_kwargs4, view_kwargs5, view_kwargs6, view_kwargs7]
for site_dir in mri_dirs_general:
    if 'fatsuppressed' not in site_dir:
        subjects_mri_directory = os.path.join(mri_dir_general,site_dir)
        subject_dirs = os.listdir(subjects_mri_directory)
        site = site_dir.split("_")[0]
        subjects_source_directory = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
        source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
        for subject in subject_dirs:
            plot_kwargs = dict(subject=subject, subjects_dir=subjects_mri_directory, surfaces='head-dense',dig=True,eeg='original')
            if subject != 'fsaverage':
                subject_reftep = subject.split("_reco")[0]
                subject_reftep_now = [name for name in os.listdir(subjects_source_directory) if subject_reftep in name]
                if len(subject_reftep_now) not in [0,1]:
                    print("Too many identifiers for this subject")
                    break
                elif len(subject_reftep_now) == 1:
                    subject_reftep = subject_reftep_now[0]
                else:
                    print(f'did not find subject {subject}/{subject_reftep} in pre-processing folder')
                    continue
                print(subject_reftep)
                sourcepath_subject = os.path.join(source_site,subject_reftep)
                coregpath_subject = os.path.join(sourcepath_subject,f'{subject_reftep}_coreg')
                if os.path.exists(coregpath_subject):
                    shutil.rmtree(coregpath_subject)
                os.mkdir(coregpath_subject)
                path_to_fif_file = os.path.join(subjects_source_directory,subject_reftep,f"{subject_reftep}_final_eeg-epo.fif")
                if os.path.exists(path_to_fif_file): #check if file exists
                    info = read_info(path_to_fif_file) #read the info
                    #start coregistration
                    coreg = Coregistration(info, subject=subject,subjects_dir=subjects_mri_directory,fiducials=fiducials)
                    coreg.fit_fiducials(verbose=True) #fit to estimated fiducials
                    fig = mne.viz.plot_alignment(info,trans=coreg.trans,**plot_kwargs)
                    figpath = os.path.join(coregpath_subject,f"{subject_reftep}-fit_estimated_fiducials.png")
                    fig.plotter.screenshot(filename=figpath)

                    coreg.fit_icp(nasion_weight=2.0,verbose=True) #fit with icp
                    fig = mne.viz.plot_alignment(info,trans=coreg.trans,**plot_kwargs)
                    figpath = os.path.join(coregpath_subject,f"{subject_reftep}-icp_fit_nasion2.0.png")
                    fig.plotter.screenshot(figpath)

                    coreg.omit_head_shape_points(distance=5.0/1000)
                    coreg.fit_icp(n_iterations=100,nasion_weight=2.0,verbose=True) #fit icp with a high number of iters
                    fig = mne.viz.plot_alignment(info,trans=coreg.trans,**plot_kwargs)
                    for view_ind, view_kwargs in enumerate(view_kwargs_all):
                        mne.viz.set_3d_view(fig, **view_kwargs)
                        figpath = os.path.join(coregpath_subject,f"{subject_reftep}-icp_aligned_electrodes_view_{view_ind}.png")
                        fig.plotter.screenshot(filename=figpath)
                    #compute some statistics
                    dists = coreg.compute_dig_mri_distances()*1e3
                    print(subject)
                    print(f"Distance between digs and MRI (mean/min/max/std)")
                    print(f"{np.mean(dists)} / {np.min(dists)} / {np.max(dists)} / {np.std(dists)}")
                    transpath = os.path.join(coregpath_subject,f"{subject_reftep}-trans.fif")
                    print(coreg.trans)
                    mne.write_trans(transpath,coreg.trans, overwrite=True)
                else:
                    print(f"did not find fif file for {subject_reftep} at {path_to_fif_file}")
